In [132]:
import numpy as np
import pandas as pd
%matplotlib inline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


from keras.models import Sequential
from keras.layers import Input, Dense, Activation, Conv2D, Flatten
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from keras import backend as K


from nltk.corpus import stopwords

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from keras import optimizers

## Обработка данных 

In [182]:
data = pd.read_csv("Roma_1.csv")

In [183]:
data = data.drop(list(data[data["sentiment"] == "IRR"].index))

In [184]:
train = data["tweet"].values[:1700]
test = data["tweet"].values[1701:]
y = data["sentiment"].values
train_cross = data["tweet"].values

In [185]:
Y = np.array(y).astype('int32')
Y = tf.keras.utils.to_categorical(y)

In [186]:
y_train = Y[:1700]
y_test = Y[1701:]

In [187]:
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         analyzer='word',
                         max_df=0.9, 
                         min_df=5
                        )

In [188]:
tf_idf_model = tf_idf.fit(train)

In [189]:
train_tf_idf_vec = tf_idf_model.transform(train)
test_tf_idf_vec = tf_idf_model.transform(test)
train_cross_vec = tf_idf_model.transform(train_cross)

## Проверка размерностей

In [190]:
y_train.shape

(1700, 4)

In [191]:
y_test.shape

(182, 4)

In [192]:
test_tf_idf_vec.shape

(182, 946)

In [193]:
train_tf_idf_vec.shape

(1700, 946)

## Создание модели

In [200]:
model = Sequential()
#model.add(Dense(461, input_dim=461))
model.add(Dense(256, input_dim=946, activation="relu"))
model.add(Dense(256, activation="relu"))
#model.add(Dense(256, activation="relu"))
#model.add(Dense(64, activation="relu"))
model.add(Dense(4,  activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[fmeasure])

In [201]:
model.fit(train_tf_idf_vec, y_train,  batch_size = 15, epochs=50, verbose=1, validation_data = (test_tf_idf_vec, y_test))

Train on 1700 samples, validate on 182 samples
Epoch 1/50
1700/1700 [==============================] - 2s 1ms/step - loss: 0.3978 - fmeasure: 0.4589 - val_loss: 0.3284 - val_fmeasure: 0.6612
Epoch 2/50
1700/1700 [==============================] - 1s 551us/step - loss: 0.2786 - fmeasure: 0.7578 - val_loss: 0.3398 - val_fmeasure: 0.6676
Epoch 3/50
1700/1700 [==============================] - 1s 539us/step - loss: 0.1987 - fmeasure: 0.8264 - val_loss: 0.3612 - val_fmeasure: 0.6739
Epoch 4/50
1700/1700 [==============================] - 1s 550us/step - loss: 0.1297 - fmeasure: 0.8944 - val_loss: 0.4502 - val_fmeasure: 0.6798
Epoch 5/50
1700/1700 [==============================] - 1s 548us/step - loss: 0.0823 - fmeasure: 0.9451 - val_loss: 0.4916 - val_fmeasure: 0.6677
Epoch 6/50
1700/1700 [==============================] - 1s 546us/step - loss: 0.0533 - fmeasure: 0.9702 - val_loss: 0.5875 - val_fmeasure: 0.6599
Epoch 7/50
1700/1700 [==============================] - 1s 600us/step - loss: 0

## Проверка результатов, метрики

In [147]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)

In [152]:
model1 = Sequential()
#model.add(Dense(461, input_dim=461))
model1.add(Dense(512, input_dim=946, activation="relu"))
#model1.add(Dense(256, activation="relu"))
#model1.add(Dense(512, activation="relu"))
#model.add(Dense(64, activation="relu"))
model1.add(Dense(4,  activation="softmax"))
sgd = optimizers.SGD(lr=0.0005, decay=0.000001, momentum=0.9, nesterov=True)
model1.compile(loss='binary_crossentropy', optimizer='sgd', metrics=["accuracy"])

In [154]:
model1.fit(train_tf_idf_vec, y_train,  batch_size = 5, epochs=20, verbose=1, validation_data = (test_tf_idf_vec, y_test))

Train on 1700 samples, validate on 182 samples
Epoch 1/20
1700/1700 [==============================] - 2s 1ms/step - loss: 0.4005 - acc: 0.8003 - val_loss: 0.3790 - val_acc: 0.7967
Epoch 2/20
1700/1700 [==============================] - 2s 1ms/step - loss: 0.3985 - acc: 0.8056 - val_loss: 0.3774 - val_acc: 0.7953
Epoch 3/20
1700/1700 [==============================] - 2s 1ms/step - loss: 0.3963 - acc: 0.8101 - val_loss: 0.3762 - val_acc: 0.8022
Epoch 4/20
1700/1700 [==============================] - 2s 1ms/step - loss: 0.3943 - acc: 0.8056 - val_loss: 0.3743 - val_acc: 0.7953
Epoch 5/20
1700/1700 [==============================] - 3s 2ms/step - loss: 0.3922 - acc: 0.8107 - val_loss: 0.3728 - val_acc: 0.8077
Epoch 6/20
1700/1700 [==============================] - 2s 1ms/step - loss: 0.3900 - acc: 0.8163 - val_loss: 0.3715 - val_acc: 0.8091
Epoch 7/20
1700/1700 [==============================] - 2s 1ms/step - loss: 0.3879 - acc: 0.8138 - val_loss: 0.3698 - val_acc: 0.8132
Epoch 8/20
1700

In [141]:
k = 10
print(model1.predict(test_tf_idf_vec)[k])
print(y_test[k])


[9.5930716e-07 7.0942442e-06 9.9974936e-01 2.4255559e-04]
[0. 0. 1. 0.]


## Cross validation

In [155]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [156]:
seed = 121
np.random.seed(seed)

In [157]:
encoded_Y = LabelEncoder().fit_transform(y)
dummy_y = np_utils.to_categorical(encoded_Y)

In [160]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(256, input_dim=946, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    sgd = optimizers.SGD(lr=0.0005, decay=0.000001, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

def baseline_model_1():
    # create model
    model1 = Sequential()
    model1.add(Dense(256, input_dim=946, activation='relu'))
    model1.add(Dense(3, activation='softmax'))
    # Compile model
    sgd = optimizers.SGD(lr=0.0005, decay=0.000001, momentum=0.9, nesterov=True)
    model1.compile(loss='binary_crossentropy', optimizer='sgd', metrics=[fmeasure, "accuracy"])
    return model1

In [161]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)
kfold = KFold(n_splits=2, shuffle=True, random_state=seed)
results = cross_val_score(estimator, train_cross_vec, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 76.37% (0.11%)


In [162]:
estimator1 = KerasClassifier(build_fn=baseline_model_1, epochs=50, batch_size=5, verbose=0)
kfold1 = KFold(n_splits=2, shuffle=True, random_state=seed)
results1 = cross_val_score(estimator, train_cross_vec, dummy_y, cv=kfold)
print("F_measure: %.2f%% (%.2f%%)" % (results1.mean()*100, results1.std()*100))

F_measure: 76.56% (0.33%)


## English data

In [168]:
angl_data = pd.read_csv("Roma_Anglichanin.csv")

In [169]:
train = angl_data["tweet"].values[:1000]
test = angl_data["tweet"].values[1001:]
y = angl_data["sentiment"].values
train_cross = angl_data["tweet"].values

In [170]:
Y = np.array(y).astype('int32')
Y = tf.keras.utils.to_categorical(y)

In [171]:
y_train = Y[:1000]
y_test = Y[1001:]

In [172]:
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         analyzer='word',
                         max_df=0.9, 
                         min_df=5
                        )

In [173]:
tf_idf_model = tf_idf.fit(train_cross)

In [174]:
train_tf_idf_vec = tf_idf_model.transform(train)
test_tf_idf_vec = tf_idf_model.transform(test)
train_cross_vec = tf_idf_model.transform(train_cross)

In [175]:
y_train.shape

(1000, 4)

In [176]:
train_tf_idf_vec.shape

(1000, 759)

In [177]:
test_tf_idf_vec.shape

(205, 759)

In [178]:
y_test.shape

(205, 4)

In [179]:
model2 = Sequential()
#model.add(Dense(461, input_dim=461))
model2.add(Dense(512, input_dim=759, activation="relu"))
#model1.add(Dense(256, activation="relu"))
#model1.add(Dense(512, activation="relu"))
#model.add(Dense(64, activation="relu"))
model2.add(Dense(4,  activation="softmax"))
sgd = optimizers.SGD(lr=0.0001, decay=0.000001, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', optimizer='sgd', metrics=["accuracy"])

In [181]:
model2.fit(train_tf_idf_vec, y_train,  batch_size = 5, epochs=20, verbose=1, validation_data = (test_tf_idf_vec, y_test))

Train on 1000 samples, validate on 205 samples
Epoch 1/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4272 - acc: 0.7820 - val_loss: 0.4626 - val_acc: 0.7537
Epoch 2/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4258 - acc: 0.7785 - val_loss: 0.4617 - val_acc: 0.7561
Epoch 3/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4244 - acc: 0.7807 - val_loss: 0.4609 - val_acc: 0.7561
Epoch 4/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4231 - acc: 0.7855 - val_loss: 0.4603 - val_acc: 0.7549
Epoch 5/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4217 - acc: 0.7872 - val_loss: 0.4597 - val_acc: 0.7549
Epoch 6/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4204 - acc: 0.7880 - val_loss: 0.4591 - val_acc: 0.7549
Epoch 7/20
1000/1000 [==============================] - 1s 1ms/step - loss: 0.4191 - acc: 0.7862 - val_loss: 0.4584 - val_acc: 0.7537
Epoch 8/20
1000

## French data

In [264]:
french_data = pd.read_csv("Roma_Franzthus.csv")

In [266]:
french_data = french_data.drop(list(french_data[french_data["sentiment"] == 99].index))

In [267]:
train = french_data["tweet"].values[:1500]
test = french_data["tweet"].values[1501:]
y = french_data["sentiment"].values
train_cross = french_data["tweet"].values

In [268]:
#Y = np.array(y).astype('int32')
Y = tf.keras.utils.to_categorical(y)

In [269]:
y_train = Y[:1500]
y_test = Y[1501:]

In [270]:
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         analyzer='word',
                         max_df=0.9, 
                         min_df=5
                        )

In [271]:
tf_idf_model = tf_idf.fit(train)

In [272]:
train_tf_idf_vec = tf_idf_model.transform(train)
test_tf_idf_vec = tf_idf_model.transform(test)
train_cross_vec = tf_idf_model.transform(train_cross)

In [273]:
y_train.shape

(1500, 3)

In [274]:
train_tf_idf_vec.shape

(1500, 741)

In [275]:
test_tf_idf_vec.shape

(382, 741)

In [276]:
y_test.shape

(382, 3)

In [279]:
model3 = Sequential()
#model.add(Dense(461, input_dim=461))
model3.add(Dense(512, input_dim=741, activation="relu"))
#model.add(Dense(256, activation="relu"))
#model1.add(Dense(512, activation="relu"))
#model.add(Dense(64, activation="relu"))
model3.add(Dense(3,  activation="softmax"))
sgd = optimizers.SGD(lr=0.0001, decay=0.000001, momentum=0.9, nesterov=True)
model3.compile(loss='binary_crossentropy', optimizer='sgd', metrics=["accuracy"])

In [280]:
model3.fit(train_tf_idf_vec, y_train,  batch_size = 5, epochs=20, verbose=1, validation_data = (test_tf_idf_vec, y_test))

Train on 1500 samples, validate on 382 samples
Epoch 1/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5940 - acc: 0.6713 - val_loss: 0.5035 - val_acc: 0.7688
Epoch 2/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5619 - acc: 0.7329 - val_loss: 0.4670 - val_acc: 0.8482
Epoch 3/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5564 - acc: 0.7320 - val_loss: 0.4540 - val_acc: 0.8482
Epoch 4/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5543 - acc: 0.7320 - val_loss: 0.4553 - val_acc: 0.8482
Epoch 5/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5523 - acc: 0.7320 - val_loss: 0.4526 - val_acc: 0.8482
Epoch 6/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5503 - acc: 0.7320 - val_loss: 0.4528 - val_acc: 0.8482
Epoch 7/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5484 - acc: 0.7320 - val_loss: 0.4506 - val_acc: 0.8482
Epoch 8/20
1500

## German data

In [298]:
ger_data = pd.read_csv("Roma_Nemets.csv")

In [308]:
ger_data = ger_data.drop(list(ger_data[ger_data["sentiment"] == "nan"].index))
ger_data = ger_data.drop(list(ger_data[ger_data["sentiment"] == "mixed"].index))[1:]

In [309]:
train = ger_data["tweet"].values[:4500]
test = ger_data["tweet"].values[4501:]
y = ger_data["sentiment"].values
train_cross = ger_data["tweet"].values

In [310]:
Y = np.array(y).astype('int32')
Y = tf.keras.utils.to_categorical(y)

In [312]:
y_train = Y[:4500]
y_test = Y[4501:]

In [313]:
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         analyzer='word',
                         max_df=0.9, 
                         min_df=5
                        )

In [314]:
tf_idf_model = tf_idf.fit(train)

In [315]:
train_tf_idf_vec = tf_idf_model.transform(train)
test_tf_idf_vec = tf_idf_model.transform(test)
train_cross_vec = tf_idf_model.transform(train_cross)

In [316]:
y_train.shape

(4500, 3)

In [317]:
train_tf_idf_vec.shape

(4500, 4530)

In [318]:
model3 = Sequential()
#model.add(Dense(461, input_dim=461))
model3.add(Dense(1024, input_dim=4530, activation="relu"))
#model.add(Dense(256, activation="relu"))
#model1.add(Dense(512, activation="relu"))
#model.add(Dense(64, activation="relu"))
model3.add(Dense(3,  activation="softmax"))
sgd = optimizers.SGD(lr=0.0001, decay=0.000001, momentum=0.9, nesterov=True)
model3.compile(loss='binary_crossentropy', optimizer='sgd', metrics=["accuracy"])

In [321]:
model3.fit(train_tf_idf_vec, y_train,  batch_size = 16, epochs=5, verbose=1, validation_data = (test_tf_idf_vec, y_test))

Train on 4500 samples, validate on 2225 samples
Epoch 1/5
4500/4500 [==============================] - 12s 3ms/step - loss: 0.4373 - acc: 0.7882 - val_loss: 0.5093 - val_acc: 0.6947
Epoch 2/5
4500/4500 [==============================] - 13s 3ms/step - loss: 0.4351 - acc: 0.7886 - val_loss: 0.5045 - val_acc: 0.6950
Epoch 3/5
4500/4500 [==============================] - 13s 3ms/step - loss: 0.4328 - acc: 0.7890 - val_loss: 0.5094 - val_acc: 0.6950
Epoch 4/5
4500/4500 [==============================] - 13s 3ms/step - loss: 0.4306 - acc: 0.7893 - val_loss: 0.5042 - val_acc: 0.6957
Epoch 5/5
4500/4500 [==============================] - 13s 3ms/step - loss: 0.4282 - acc: 0.7898 - val_loss: 0.4977 - val_acc: 0.6966
